## The purpose of this report : 

Is to try to determin if a marital status of a customer, and the number of childrenhe of a customer, have an effect on the default of of the customers loan.This based on the data collected by the credit devision. The report will be considered when building the **credit score** of a potential customer.

## First thing first:

We start by see what we have. What information we got, what its quality.the data lies in the credit_scoring_eng.csv file. We need to read the data to see what we are deeling with. What is in this file and how can it help us in our task of trying to determin if the personal status of the customer has a on it ability to return the loan.


In [1]:
# Loading all the libraries
import pandas as pd
import numpy as np

# Load the data
#the credit_df veriable, will be used for storing the data from the CSV file we got.
#we will use try-except to be able to read the file also if it is localy saved

try:
    credit_df = pd.read_csv('/datasets/credit_scoring_eng.csv') #platform saved
    
except:
    credit_df = pd.read_csv('credit_scoring_eng.csv') #localy saved

credit_df.head() # printing the first 5 rows to see the load went ok and we got the data we were expecting 
                #This print is without a python print() command due to the nature of the print format, that is
                # more readable, then if I used the command - probably due to the nature of the Jupiter notebook


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


## Task 1. Data exploration

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


    **The data in now loaded and we need to proces it for analyzing. First we will check the structure of the table,then we will need to look on  column lables to see if there are any issues we need to fix. after that we will need to deal with missing and duplicated values - if we will see that tere are any.

In [2]:
# Let's see how many rows and columns our dataset has
#using the shape methode will give us the idea of how much data are we dealing with? Is this database large 
#enough to get any conclusion from.

credit_df.shape

(21525, 12)

We have 21,525 records of raw data. Lets print a sample to have some bettre look on potential issues
But there is One thing we must do Before we go on is to look at the column names, to see ant structural issues.
just to make sure.

In [3]:
credit_df.columns # printing the column header to look for any structural of miss_type errors

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Examining the header, shows that there are no structural nor spelling or typing errors. So we can proceed to examit the data it self.

In [4]:
# let's print the first 100 rows of the credit_df table

credit_df.head(100) #since we have such a large database we nead a wider look - an less then the firs 100 rows will
                    #not do, in order to scan for potential issues with the raw data we got.



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,-541.832241,51,bachelor's degree,0,married,0,F,employee,0,15070.060,car
96,0,NaN,44,SECONDARY EDUCATION,1,married,0,F,employee,0,NaN,buy residential real estate
97,0,NaN,47,bachelor's degree,0,married,0,F,employee,0,NaN,profile education
98,0,364906.205736,54,bachelor's degree,0,married,0,F,retiree,0,31953.168,buying property for renting out


The first thing that strikes the eye id, that we have negative values in the 'days_employed' column, where all values should be positive(If a person worked even only one day in its life , the value is at least 1 and if he never worked the value should be 0) 
This brings me to the second issue, we can detect - missing values in the  'days_employed' and 'total_income' columns represented by NaN values. we will need to get more information, because we will need to prepare the data and "clean" it befor we can use it.

In [5]:
# Get info on data using the info() function will give us more detailed information on the distributon of the data,
#its data type structure and uniformity  of the values.
credit_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


From a glance in the data we can see that both 'days_employed' column and 'total_income' column , have the same amount of missing values. although it istha same amount we can not draw a conclusion that there is a symmetry in the rows where they appear. we will need to filter the data where there are null values, and see  if they consist with the number of missing values in both columns

In [6]:
# Let's look in the filtered table at the first column with missing data 
#using isnull() will return all the data that has missing values. 
#the len_miss_data will save the number of missing values we found. 

credit_df_days_null = credit_df.loc[(credit_df['days_employed'].isna())] # filterin the data of 'days_employed'
                                                                        #with null values to new credit_df_null veriable

len_miss_data = len(credit_df_days_null) # the number of records with the null value in 'days_employed' 
print(credit_df_days_null.iloc[0]) #printing the first row with the missing data
print()
print('\n**The number of records in the "days_employed" column is: {}'.format(len_miss_data)) #print the number of records that this filtered data has


children                              0
days_employed                       NaN
dob_years                            65
education           secondary education
education_id                          1
family_status         civil partnership
family_status_id                      1
gender                                M
income_type                     retiree
debt                                  0
total_income                        NaN
purpose               to have a wedding
Name: 12, dtype: object


**The number of records in the "days_employed" column is: 2174


when we look on the first row with missing values from 'days_employed' column, we can see tat on the same line the 'total_income' has also a null value so in this case there is symetry between the two columns.
If we want to see if all the null values are symetrical, in bothe columns , me must show that the same number of rows is consistent in both columns query snd the single column query ('total_income' and 'days_employed' columns) if we will see that we have the same amount of missing values,then we can conclude that there is a symetry in both columns, with the missing values, meening where there is a missing value in 'total_income' column, we will see in the same row a missing value in the 'days_employed' column, and vers versa.

**Intermediate conclusion**

Our immidiate conclusion is that The numbe of rows is shows that there is a symetry on both "days_employed" column and "total_income" ,is the same and from that we can conclude that there is a symetry in bothe columns where there are missing values.

The total percantage of the missing values in the table is there for 9.97% almst 10% of the data.
It is not that considrable part of the data(we have 19,351 records with of data), and if we will make a conclusion that the nature of these rows will not have an impact on our final assesment we can delete them all togather and move on, to check the remaining data.

we will need to check now the characteristic of the clients that have no values in these columns.
we already mde the 'credit_df_null veriable to obtain all the records with the missing values. let us now check if there is somthing common with the customers' data.

if there is somthing incommon we will not be able to delet these rows and will have to fill the missing values. But if we will get to a conclusion that there is no common ground to the records, and there is seems to be a random resone for the missing values, we can then considre deleting the rows and moov on with the data checking.

But First - we must print some of the rows with the missing data to see if there is an obvious pattern.


In [7]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values
credit_df_null = credit_df[credit_df.isna().any(axis = 1)]

credit_df_null.head(50) # print the firstb 50 rows


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing


In [8]:
credit_df_null.tail(50) # Print the last 50 rows

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21000,2,NaN,35,secondary education,1,married,0,M,employee,0,NaN,purchase of the house
21031,0,NaN,56,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21057,0,NaN,62,secondary education,1,civil partnership,1,M,employee,0,NaN,wedding ceremony
21060,0,NaN,23,secondary education,1,unmarried,4,F,employee,0,NaN,construction of own property
21070,0,NaN,59,bachelor's degree,0,married,0,F,retiree,0,NaN,to become educated
21077,1,NaN,41,secondary education,1,married,0,F,business,0,NaN,buy real estate
21085,0,NaN,48,secondary education,1,civil partnership,1,F,employee,0,NaN,housing transactions
21090,0,NaN,41,some college,2,married,0,M,business,0,NaN,housing renovation
21102,0,NaN,60,secondary education,1,married,0,M,employee,0,NaN,building a property


The first thing we can notice that almost all the missing data has a common three values in the 'education' column.The values 'secondary education', 'bachlor's degree' and 'some collage are repeted in the 'education' column where there are missing values. But in all the other columns it seems that there is no common values and is distributed randomly.
lets check each column distribution ov values to see if there are repeted values.

In [9]:
# Checking distribution ov values for every column on both general data and missing value data

print('\n{} column:\n'.format('education_id'), credit_df['education_id'].value_counts()) # column education_id
print('\n{} missing data column:\n'.format('education_id'), credit_df_null['education_id'].value_counts()) # column education_id 

print('\n{} column:\n'.format('dob_years'), credit_df['dob_years'].value_counts()) # column dob_years
print('\n{} missing data column:\n'.format('dob_years'), credit_df_null['dob_years'].value_counts()) # column dob_years

print('\n{} column:\n'.format('family_status_id'), credit_df['family_status_id'].value_counts()) # column family_status_id
print('\n{} missing data column:\n'.format('family_status_id'), credit_df_null['family_status_id'].value_counts()) # column family_status_id

print('\n{} column:\n'.format('gender'), credit_df['gender'].value_counts()) # column gender
print('\n{} missing data column:\n'.format('gender'), credit_df_null['gender'].value_counts()) # column gender

print('\n{} column:\n'.format('income_type'), credit_df['income_type'].value_counts()) # column income_type
print('\n{} missing data column:\n'.format('income_type'), credit_df_null['income_type'].value_counts()) # column income_type

print('\n{} column:\n'.format('purpose'), credit_df['purpose'].value_counts()) # column income_type
print('\n{} missing data column:\n'.format('purpose'), credit_df_null['purpose'].value_counts()) # column income_type




education_id column:
 1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

education_id missing data column:
 1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64

dob_years column:
 35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

dob_years missing data column:
 34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55


**Possible reasons for missing values in data**

Looking breifly on the both tables,the reason comes in mind that there is probably a technical problem that caused this values from being missed.The symetry in two colums in the same indexes seems to be randomly picked.  

one reason can be missing supporting documents - 
But the consistency an symentry in both columns is not 'natural'.A customer can have on or two missing documents each for the column with the missing values, But that would have to make a-symetry in missing values. where on customer did not aapliied one document and the other customer did not applied both documents.there for makink an non-symetrical missing values patern. We do not see it here so we can exclude this resone

We will try to look on the next steps to see if, we can support this hypotheses, looking if there are other values of the other columns repeting in a patern or not(will we fine one, two or more values repeating where there are missing values) , in the filtered  table.
Building a loop that will print the a comperison between the number of unique values that are in every column of both the general data and the filltered data if the give a sense if there is a patern or almost all the values apears in the filtered data.If we sea a major decreas in on of the columns unique values to appear in the filltered data, we will investigate the values that will appear , for further investigation. If there will be a small or non decrese, then we can exclude a patern in the missing data values.

Since there are the missing values in the 'days_employed' and the 'total_income' columns, and the 'education_id' and 'family_status_id' are the number-value image of the 'education' and 'family_status' columns values, we will not include them in our check.



In [10]:
credit_df.columns # printing the columns names for extracting the relevent ones 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [11]:
#verables  for the function
#list of the name of columns
lst_columns = ['children','dob_years','education','family_status','gender','income_type','debt','purpose'] 

for clmn in lst_columns:
    
    print()
    print('unique values in the- {} -column:'.format(clmn))
    print('full data unique values shows: ',len(credit_df[clmn].unique()))
    print('filtered data unique values shows: ',len(credit_df_null[clmn].unique()))
    


unique values in the- children -column:
full data unique values shows:  8
filtered data unique values shows:  8

unique values in the- dob_years -column:
full data unique values shows:  58
filtered data unique values shows:  56

unique values in the- education -column:
full data unique values shows:  15
filtered data unique values shows:  12

unique values in the- family_status -column:
full data unique values shows:  5
filtered data unique values shows:  5

unique values in the- gender -column:
full data unique values shows:  3
filtered data unique values shows:  2

unique values in the- income_type -column:
full data unique values shows:  8
filtered data unique values shows:  5

unique values in the- debt -column:
full data unique values shows:  2
filtered data unique values shows:  2

unique values in the- purpose -column:
full data unique values shows:  38
filtered data unique values shows:  38


**Final conclusion on the patern question we seeked.**

The values that are shown, dissmiss the posibility of a patern.
Almost all the values that show in the original data, are also in the filtered data.
There for we can derive that there is no obviouse patern where there are missing values.
This fact, strengthen the evaluation that these values are due to a technical problem.


**The missing values effect on the general data**

As for these missing values, we will now see how much every education of  its missing 'total_incom' values, effect on the general education_id, 'total_income' missing values(what is the percentaged of missing values on every education_id and its 'total_income' value), on values from the general data. (we will use the 'education_id' column to simplify our search)
We will use the calculated effect (prcentage based on the mean of the total incom), to decide on our next step and how we will relate to the missing values.

As of the base that we will take as a guide for further actions: 
-If the effect we calculated is up to 3% of the total income, we can delete the records,since this is a plausable human error efect, that we can take in acount. 
-Any other effect we will have will make us complete the missing data and moove on to the next step of prepering the data.


In [12]:
# Checking the distribution in the whole dataset by calculating the percentage of the missing values in the
#education_id column and the total_income column and what is the effect on the mean and median values we get
#in the total_income column 

#veriables for the calculation process
edu_total_unique = credit_df['education_id'].unique() #unique values of the columns
edu_missing_val_unique = credit_df_null['education_id'].unique()

edu_total = credit_df['education_id'].value_counts() #numbe of rows in this column of the full data set
edu_missing_val = credit_df_null['education_id'].value_counts()#numbe of rows in this column of the filtered data set

incom_total =  credit_df[['education_id','total_income']] 

#dictionary for translating education_id value to its text actual value - will be used in the print code
edu_dict = { 0:'bachelors degree' , 1:'Secondary Education' , 2:'some college' , 3:'primary education' , 4:'Graduate Degree'}
                         
#calculate the % of each education_id - using a for loop

for edu_id in edu_total_unique: # working through the list of unique values in the full data set
    if edu_id in edu_missing_val_unique: #checking if this value is also in the filtered data values
        
        edu_id_missing_percent = edu_missing_val[edu_id] / edu_total[edu_id] #calculating the percentage of 
                                                                                #each existin value
        print('\nThe precentage of missing values in the {} in education column is: {:.1%}'.format(edu_dict[edu_id],edu_id_missing_percent))
    
        med_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].median()
        mean_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].mean()
        
        
        #calculating the pecentage effect of total missing records on the total income
        
        total_records = len(credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)]) #total records in the full table
        total_incom_of_records = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].sum()# total incom in the full table
        
        number_of_mising_records = len(credit_df_null['total_income'].loc[(credit_df_null['education_id'] == edu_id)]) #number of records of missing records
        
        based_mean_income_missing = number_of_mising_records * mean_total_income # total income missing based on the mean of total incom
        total_income_based_mean =  total_incom_of_records + based_mean_income_missing # the total incom we have had if there were no missing values
        print('The total of records are :  {}'.format(total_records))
        print('The total of income of nun null records is :  {}'.format(total_incom_of_records)) # the summ of total incom in the full table
        print('The total missing records are :  {}'.format(number_of_mising_records)) # number of missing records
        print('The total income "($)" missing based on mean of {} is: {}'.format(edu_dict[edu_id] , based_mean_income_missing))# the sum of income if no missing values were present
        print('The effect on the Total incom is an increase of : {:.0%}'.format((1-(total_incom_of_records/total_income_based_mean)))) #the effect the missing incom has on the calculated total
    
    else:
        edu_id_missing_percent = 0
        try: 
            med_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].median()
            mean_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].mean()
        except:
            med_total_income = -1
            mean_total_income = -1
        print('\nThe precentage of missing values in the {} in education column is: {:.1%}'.format(edu_dict[edu_id],0))
        print('The mean and Total were not calculated')
    
    


The precentage of missing values in the bachelors degree in education column is: 10.3%
The total of records are :  5260
The total of income of nun null records is :  156301456.27899998
The total missing records are :  544
The total income "($)" missing based on mean of bachelors degree is: 18029684.52412553
The effect on the Total incom is an increase of : 10%

The precentage of missing values in the Secondary Education in education column is: 10.1%
The total of records are :  15233
The total of income of nun null records is :  336772530.087
The total missing records are :  1540
The total income "($)" missing based on mean of Secondary Education is: 37875534.677132845
The effect on the Total incom is an increase of : 10%

The precentage of missing values in the some college in education column is: 9.3%
The total of records are :  744
The total of income of nun null records is :  19605674.46
The total missing records are :  69
The total income "($)" missing based on mean of some colleg

**Intermediate conclusion**

After I have made a comparacent between the two tables, the full table and the filterd table , I founf no pattern, that will indicate of an intentional cause for the missing values.
There is no patern of other values that are cosistent with the missing values we have.Almost all other values of all other columns, appear in the filtered data, and not with any obvious patern.
 
There for my conclustion is that there is or was, probably a technical problem that caused this values of being missed.
The second reason I conclude this is a technical problem ,is due to the nature of the missing values.
You would expect that the total income of a customer , and its potential hyring ability (as an effect of his working days represented in 'days_employed' column), would be taken in consideration, when applying a loan. 
 reason is, that 
**There for the coclusion is that there was a technical problem causing the missing values.

**So I sugest thet the system tech personal, will recieved this information for further investigation on theire part.


**As for the missing values themselves ** 

Since the effect maxed the 3% (see above comperison of data) that we have made as an guide for deciding waht our next step , dealing with the missing values, We will need to fill the missing values by the mean or median of each missing row value - and we can use the mean and median we extracted in the previouse check for the effect of the missing values, to use it and add the 'days_employed' column fron this values to be processed.

**The next step**

After deciding that we need to fill the missing data, we need to determin with what values.
Let us get the mean and median values of both 'days_employed' and 'total_income columns.
we will do that, based on their education, that is a good referance to go by, to fine in the same educational group the mean and median of their total income.We can assume that the change of the salerries in these groups will not differ much.
If the diffrance between median and meen , of the educational group , will be very distinct, we will use the median value.If not, then we can use the mean.

**Just on last thing befor we go on**
We have a problem in the 'days_employed' column that we detected erlier in our check. 
The column have *negative values*. 
we will change the negative values that looks like a human error misstake, back to positive values befor we will calculate the mean and median of this column.





In [13]:
#using abs() function to replace all values in the days_employed to a positive values
#and then check if all were chenged - if they were the printed resauls should be 0.

credit_df['days_employed'] = abs(credit_df['days_employed']) 
print(len(credit_df.loc[(credit_df['days_employed'] < 0)]))

0


In [14]:
#getting the mean and median for each education_id
#using dynamic dictionary

def mean_median_find(edu_total_unique):

    med_mean_dict = {}
    
    for edu_id in edu_total_unique: # working through the list of unique values in the full data set
        #each veriable will return the mean and medin in both total_income and days_employed columns 
        med_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].median()
        mean_total_income = credit_df['total_income'].loc[(credit_df['education_id'] == edu_id)].mean()
        med_days_employed = credit_df['days_employed'].loc[(credit_df['education_id'] == edu_id)].median()
        mean_days_employed = credit_df['days_employed'].loc[(credit_df['education_id'] == edu_id)].mean()
        med_mean_dict[edu_id] = [[med_total_income,mean_total_income],[med_days_employed,mean_days_employed]]
    
    return(med_mean_dict)

edu_unique  = credit_df['education_id'].unique() # list the unique education id values

median_mean_dict = mean_median_find(edu_unique) #calling te function to retrieve the mean and median values for each column
                                                # based on education_id column

#dictionaries that will contain the mean and median for each education id of 'total_income' column
#These veriables will be used leter to fill the missing values

income_mean_dict = {}
income_median_dict = {}

for edu_id in credit_df['education_id'].unique():
        
    print ('for education Id:',edu_id)
    print ('total_income column median : ',median_mean_dict[edu_id][0][0])
    income_median_dict[edu_id] = median_mean_dict[edu_id][0][0]
    print ('total_income column mean : ',median_mean_dict[edu_id][0][1])
    income_mean_dict[edu_id] = median_mean_dict[edu_id][0][1]
    print ('for education Id:',edu_id)
    print ('days_employed column median : ',median_mean_dict[edu_id][1][0])
    print ('days_employed column mean : ',median_mean_dict[edu_id][1][1])




print('\n')
print(income_mean_dict)
print(income_median_dict)

    
    

for education Id: 0
total_income column median :  28054.531000000003
total_income column mean :  33142.80243405428
for education Id: 0
days_employed column median :  1895.7477953984824
days_employed column mean :  42375.40917437274
for education Id: 1
total_income column median :  21836.583
total_income column mean :  24594.50303709925
for education Id: 1
days_employed column median :  2392.4835002975465
days_employed column mean :  76413.82237201057
for education Id: 2
total_income column median :  25618.464
total_income column mean :  29045.443644444447
for education Id: 2
days_employed column median :  1209.1280832366251
days_employed column mean :  20656.63201666237
for education Id: 3
total_income column median :  18741.976
total_income column mean :  21144.88221072797
for education Id: 3
days_employed column median :  3043.933614681601
days_employed column mean :  130340.42634924148
for education Id: 4
total_income column median :  25161.5835
total_income column mean :  27960.024

**Intermediate conclusion**

The resaults show two things:

1- The mean and median of all the education id groups are almost the same there for we will use in the 'total_income' column the, mean to change the values.

2 - In the 'day_employed' there are big differences between the mean and median values.so here we will use the median as our value.

We can now use the dictionaries we made in the previouse section to fill in the missing data acording to the values we decided are the most appropriate.
For the 'total_income' missing values we will use the income_mean_dict veriable as the referance for the values to change to. And for the 'days_employed' column missing values we will use the median as referance.


**Conclusions**

This Part showed that the missing values have no patteren, that suggest that the values were intentionaly missed, due to some value cretiria. They appeared randomaly but in consistence with each other in both 'days_employed' and 'total_income' columns.
The lack of ducumants for each category('dayes employed' and 'total_income' columns), was dissmised due to the full symetry in both columns.
After looking at the data, the acumelated evidance suggested a system technical problem.
Since this was the case and the check of the missing data showed that the values must be filled, we have made the preparations to fill all missing data according to the education_id a customer have, that will give us a common ground to devide the customers to groups that are most likely to be payed the same amount.

**before proceeding filling the missing values we will looke on the other columns to see if there are any issues there to fix ***

## Data transformation


In this section we will try to address any issues of columns values in terms of wrong values, diffrent data format, spelling problems and so on.


#### Checking the 'education' column.

In [15]:
# Let's see all values in education column to check if and what spellings will need to be fixed
#w will begin by printing the unique value of the education column, to look for any misstypes or other problems

credit_df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

There are only five categories of education_id numbers. But much more education types. most ofthem are the duplicates in a diffrent format.
We will use the apply method to make the canges, all values will become lower-case format.

In [16]:
# Fix the registers to cahnge all the enteries to one lower case format. 
#That way making all duplicated other formats disapear.

credit_df['education'] = credit_df['education'].str.lower()
                                                    

In [17]:
# Checking all the values in the column to make sure we fixed them
credit_df['education'].unique()


array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

#### Check the data the `children` column

In [18]:
# Let's see the distribution of values in the `children` column
credit_df['children'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5])

We see two unique values that simms out of place:

(-1 , 20).

The value of 20 is possible though unlikely, and the value of -1 is definatly a miss type error. You can not hav -1 childe. You have children or you do not have children.

let see the rows they appear in, to see if we can draw a conclusion on them.


In [19]:
#printing the lines with thos values for inspection
credit_df.loc[(credit_df['children'] == -1) | (credit_df['children'] == 20)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417.703588,46,secondary education,1,civil partnership,1,F,employee,0,16450.615,profile education
606,20,880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
705,-1,902.084528,50,secondary education,1,married,0,F,civil servant,0,22061.264,car purchase
720,20,855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
742,-1,3174.456205,57,secondary education,1,married,0,F,employee,0,10282.887,supplementary education
...,...,...,...,...,...,...,...,...,...,...,...,...
21140,-1,1422.668059,44,secondary education,1,married,0,F,business,0,27129.935,transactions with my real estate
21325,20,601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


There are 123 rows that have these problem. 
The 'dob_years'(age of the customer) value verries.
But one age stands : 21. 
For this customer the number of children value is: 20. Meanning he would have to have his first child when he was 1 year old. not possible.
There for the real number of children must be: 2. 
And the real value for the -1 should be: 1.
We will aplly it too the column.

In [20]:
#changing the values of the children column that have the -1 and 20 values.

credit_df['children'].replace(to_replace = -1 ,value = 1 , inplace = True)
credit_df['children'].replace(to_replace = 20 ,value = 2 , inplace = True)


In [21]:
# Checking the `children` column again to make sure it's all fixed
credit_df['children'].unique()


array([1, 0, 3, 2, 4, 5])

#### Checking 'days_employed' column values

The 'days_employed', column we have already saw , that there are missing values in it. And also had a problem with negative values, which we already approched previously.
Let see if there are other issues that needsto be addresed.



In [22]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentag
#print(credit_df['days_employed'].unique())

credit_df['days_employed'].unique()


array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

I See no other problems in the 'days_amployed' column, except the missing values that we will address now.
We alreaady have decided on the way we will aproch this matter, by using the median to fill each missing value. This is based on the check of the values previously. And will address them further on.

#### Checking the 'dob_years' column

Let's now look at the client's age and whether there are any issues there.

In [23]:
# Check the `dob_years` for suspicious values and count the percentage

print(credit_df['dob_years'].sort_values().unique())


[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


In the values there is 0 y/o person. 
First let us see the rows it appears. And then decide how to approch thise value.

In [24]:
# Address the issues in the `dob_years` column, if they exist
#Printing the rows with the value 0
credit_df.loc[credit_df['dob_years'] == 0]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,secondary education,1,married,0,F,retiree,0,11406.644,car
149,0,2664.273168,0,secondary education,1,divorced,3,F,employee,0,11228.230,housing transactions
270,3,1872.663186,0,secondary education,1,married,0,F,employee,0,16346.633,housing renovation
578,0,397856.565013,0,secondary education,1,married,0,F,retiree,0,15619.310,construction of own property
1040,0,1158.029561,0,bachelor's degree,0,divorced,3,F,business,0,48639.062,to own a car
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,secondary education,1,married,0,F,employee,0,NaN,housing
20462,0,338734.868540,0,secondary education,1,married,0,F,retiree,0,41471.027,purchase of my own house
20577,0,331741.271455,0,secondary education,1,unmarried,4,F,retiree,0,20766.202,property
21179,2,108.967042,0,bachelor's degree,0,married,0,M,business,0,38512.321,building a real estate


There Are 101 rows with this problem. and there is no way we can now the age of every person. some are retiered meanning their age is over 65 and some are employed or own business.

Also, there are only 101 records that coplies 0.4% of the general data. There for we can delete these records,and save time on these issues.

In [25]:
credit_df.drop(credit_df.index[credit_df['dob_years'] == 0] , inplace = True)

In [26]:
# Check the result - make sure it's fixed
#The value that should be returned is 0 if we deleted all the rows
print(len(credit_df.loc[credit_df['dob_years'] == 0]))
#checking the values destribution now
print(credit_df['dob_years'].sort_values().unique())

0
[19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75]


#### Checking the 'family_status' column

Now let's check the `family_status` column. See what kind of values there are and what problems we may need to address.

In [27]:
# Let's see the values for the column
credit_df['family_status'].unique()


array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

There are no apperent issues with the values in the 'family_status' column

#### Checking 'gender' column values

[Now let's check the `gender` column. See what kind of values there are and what problems you may need to address]

In [28]:
# Let's see the values in the column
credit_df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

We see an unfemilier value 'XNA'. let us see what records appear with this value.

In [29]:
# Address the problematic values, if they exist
#printing the records with XNA in them

credit_df.loc[credit_df['gender']=='XNA']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


There is only one record and we can delete it this will not effect our furter investigation.

In [30]:
# Check the result - make sure it's fixed
#delete the row
credit_df.drop(credit_df.index[credit_df['gender'] == 'XNA'] , inplace = True)
#check the result - if deleted - will not apear in the values printed
credit_df['gender'].unique()


array(['F', 'M'], dtype=object)

#### Checking the 'income_type' column values

Now let's check the `income_type` column. See what kind of values there are and what problems need to address

In [31]:
# Let's see the values in the column
credit_df['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

No issues found in this column.



### Chekcing duplicate rows

It is time to check the duplicated data and delete these rows


In [32]:
# Checking duplicates
credit_df.loc[credit_df.duplicated()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


There are 71 rows of duplicated data ,so we can delete them

In [33]:
# Address the duplicates, if they exist
#delete the duplicated rows

credit_df.drop_duplicates(keep=False, inplace=True, ignore_index=True)

In [34]:
# Last check whether we have any duplicates
# if we have deleted all duplicates , we will get an empty table
credit_df.loc[credit_df.duplicated()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


We have addresed al structural issues of the columns. and now the size of our data has changed. We will now look at the new dimensions of the data.

In [35]:
# Check the size of the dataset that you now have after your first manipulations with it
credit_df.shape

(21286, 12)

**In conclusion of all the chages we made in all the columns, we see that the number of records droped.
The number of the records now is 21286 , and the original number of records was 21525 records.
The difference is 239 records wich are only 1.1% of the original data. 
So we can now proceed to deal with the missin values of the data.**


# Working with missing values

The missing data was already obtained and the decisions of how to approched it was delt and prepared, Earlier.
As a reminder, We have constructed for the 'total_income' column a dictionary called 'income_mean_dict', where we have saved the mean of total_income according to the education_id of the customer.Each id has its mean value and with these values we will fill in the missin values, of'total_income' column.

As for the 'dayes_employed' column, we can make a new column that will categorize the values by age groups, giving us two advantages:

1 - the days_employed median that we will apply , will be more close to the real days , because of the number of working days the customer could have gained based on its age( a 21 y/o customer has less working day then a 45 y/o customer)

2- The categories by age can help in our query on the ability to return a loan.

To categorize the data by group ages we will devide the ages to 7 groups(19-20,21-30,31-40,41-50,51-60,61-70,70+)

we will build a function that will return the age category and apply it to the 'dob_yeas', culomn, creating a new 'age_category' column
we will also make a dictionary that will be the interpeter of the age_catecory (marked 1-7).



In [36]:
# Find the dictionaries
# we already have the income_mean_dict - that has the means education based. Now we will build the dictionary 
#for the age groups

age_group_dict = {1:'19-20' , 2:'21-30',3:'31-40',4:'41-50',5:'51-60',6:'61-70',7:'75+'}

In [37]:
# Making a function that will halp us with identyng the incoe type of a customer. I will use the unique 
#to creat the list from whiche i will make a dictionary for the categories I will make for the income type.

income_type_lst = list(credit_df['income_type'].unique())

#making the dynamic dictionary based on the income type:
#'employee', 'retiree', 'business', 'civil servant', 'unemployed', 'entrepreneur', 'student', 'paternity / maternity leave'
income_type_dict = {}

for income in income_type_lst:
    if income == 'entrepreneur':
        income_type_dict[income] = 1
    elif income == 'business':
        income_type_dict[income] = 2
    elif income == 'employee':
        income_type_dict[income] = 3
    elif income == 'civil servant':
        income_type_dict[income] = 4
    elif income == 'student':
        income_type_dict[income] = 5
    elif income == 'retiree':
        income_type_dict[income] = 6
    elif income == 'paternity / maternity leave':
        income_type_dict[income] = 7
    elif income == 'unemployed':
        income_type_dict[income] = 8

#a function that will return the numeric value of the category of the 
def retun_income_grade(income):
    return(income_type_dict[income['income_type']])

#apply to income type column

credit_df['income_type_grade'] = credit_df.apply(retun_income_grade , axis = 1)

#checking the values in the new column
credit_df.head(20)
    
    


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,3
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,3
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,3
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,6
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,2
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,3
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,3
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,3


### Restoring missing values in `total_income`

As stated above the total income will be restored by approaching the education id of the customer.
But first let us build the age category that will serve us later on in restoring the days_employed column.



In [38]:
# Let's write a function that calculates the age category
def age_categorize(age_val):
    
    if age_val<=20:
        
        return (int(1))
    
    elif age_val >= 21 and age_val<=30:
        
        return (int(2))
    
    elif age_val >= 31 and age_val<=40:
        
        return (int(3))
    
    elif age_val >= 41 and age_val <=50:
        
        return (int(4))
    
    elif age_val >= 51 and age_val<=60:
        
        return (int(5))
    
    elif age_val >=61 and age_val<=70:
        
        return (int(6))
    
    elif age_val >= 71 :
        
        return (int(7))
    
    
    
    
    

In [39]:
# Test if the function works
print(age_categorize(18))
print(age_categorize(33))
print(age_categorize(52))
print(age_categorize(81))

1
3
5
7


In [40]:
# Creating new column based on function
credit_df['age_group'] = credit_df['dob_years'].astype('int').apply(age_categorize)
#creating new dataframe containing only the null Values for a comperison resons
credit_df_null_new = credit_df[credit_df.isnull().any(axis = 1)]

In [41]:
# Checking how values in the new column
credit_df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,3,4
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,3,3
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,3,3
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3,3
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,6,5
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2,2
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,2,4
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,3,4
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,3,3
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,3,4


In [42]:
credit_df_null_new.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,6,6
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,4,4
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,6,6
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,4,4
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,6,5
65,0,NaN,21,secondary education,1,unmarried,4,M,business,0,NaN,transactions with commercial real estate,2,2
67,0,NaN,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family,6,5
72,1,NaN,32,bachelor's degree,0,married,0,M,civil servant,0,NaN,transactions with commercial real estate,4,3
82,2,NaN,50,bachelor's degree,0,married,0,F,employee,0,NaN,housing,3,4
83,0,NaN,52,secondary education,1,married,0,M,employee,0,NaN,housing,3,5




Most salaries are based on the way a man earns his leaving(shown by 'income_type'column), and the seniority(days_employed), of that person in the type of income he has.

The salleries are also based on the education a man has, but because it can varie in the same income type, we can exclude it as a parameter we will take in considaration, when extracting the total_income and days_employed values, that we will then manipulate to get the values of the missing values.

since we have both days_employed and 'total_income' columns, have missing values , I think that categorizing the values of both columns under age category, will give us the details for both columns that we need.


**But the first step**

We will start by extracting the data we already have to a new table.
A table without the missing values. so we can get te mst information depending on the full data we have.

In [43]:
# Create a table without missing values and print a few of its rows to make sure it looks fine

credit_df_nomiss = credit_df.dropna()
credit_df_nomiss.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,3,4
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,3,3
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,3,3
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,3,3
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,6,5
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,2,2
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,2,4
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,3,4
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,3,3
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,3,4


**Next step**

aggragate the values of both 'days_employed' and 'total_income' under the 'age_group',and 'income_type_grade' categories , with both means and medians, of these values, for each age group and income type.



In [44]:
# Look at the mean values for income based on the age_group and income_grade parameters

total_incom_mean_age_inc = credit_df_nomiss.groupby(['age_group','income_type_grade']).agg({'days_employed':['mean','median'],'total_income':['mean','median']})

total_incom_mean_age_inc

days_employed                 total_income  \
                                      mean         median          mean   
age_group income_type_grade                                               
1         2                     515.985625     542.867211  20223.495136   
          3                     782.125220     868.770587  19040.474088   
          4                     600.437357     509.969922  21099.632667   
2         1                     520.848083     520.848083  79866.103000   
          2                    1222.256221     982.308535  29547.952832   
          3                    1271.615129    1059.334917  24299.975239   
          4                    1709.824672    1492.890520  25218.280307   
          5                     578.751554     578.751554  15712.260000   
          6                  360706.968329  364348.197352  14888.651857   
3         2                    1900.201337    1526.265196  33068.546597   
          3                    2069.753545    1568.383144  26347.145920   
          4                    2954.475076    2709.926313  28242.827136   
          6                  366218.199639  366120.685484  22164.904028   
          7                    3296.759962    3296.759962   8612.661000   
          8                  337524.466835  337524.466835   9593.119000   
4         2                    2506.981092    1944.345291  33982.135797   
          3                    2778.171800    1977.445600  26052.497606   
          4                    4030.836455    3434.535923  28037.978307   
          6                  365482.830195  366524.770381  26492.848641   
          8                  395302.838654  395302.838654  32435.602000   
5         2                    2899.605957    2051.793434  32431.747401   
          3                    3271.309661    2235.163052  26207.542728   
          4                    4944.394327    4027.073137  26404.586045   
          6                  364620.538773  364476.428216  21981.722046   
6         2                    3644.075107    2358.275817  32668.898026   
          3                    3883.335727    2669.073965  27825.431709   
          4                    4390.716537    3318.440092  30713.052864   
          6                  365290.737157  365718.329655  21536.347168   
7         2                    4622.013775    3095.344969  29211.780833   
          3                    4266.205758    4093.896003  23320.184000   
          4                    1204.922098    1204.922098  19530.677500   
          6                  367015.731064  367877.010970  18730.020326   

                                         
                                 median  
age_group income_type_grade              
1         2                  19648.2745  
          3                  16356.9390  
          4                  12125.9860  
2         1                  79866.1030  
          2                  25777.2835  
          3                  21916.2890  
          4                  23300.0970  
          5                  15712.2600  
          6                  12807.0710  
3         2                  28969.5790  
          3                  23262.4510  
          4                  25015.2790  
          6                  18475.1010  
          7                   8612.6610  
          8                   9593.1190  
4         2                  28352.4130  
          3                  22952.0400  
          4                  23847.2850  
          6                  22740.0690  
          8                  32435.6020  
5         2                  27348.5870  
          3                  22696.3680  
          4                  23392.7575  
          6                  19258.8770  
6         2                  29171.9890  
          3                  23550.5120  
          4                  25144.1470  
          6                  18422.5430  
7         2                  28138.8950  
          3                  20246.1230  
          4      

**intemediate decission**

After I looked at the data fro the aggregation of the data by the two factors of, age group and income grade, I see that the data distributes a bit diferantly between the mean and the median. 
Because the more accurate destribution of values is considred to be the median, I decided to go with the median values , which I will use to fill the missing values of bothe columns.



We see from this comperisons of diffrent agegroups and the education id they are in , that there is a constat rise of the income (bothe mean and median), up to age group no. 4 (41-50). and after that there is a decline in these age groups of 5 to 7 (51-75+).

let see the age verses gender , to watch the diffrences , maybe we will need to take age and gender as our factors in determaning the value each missing value willget.


**I want to check befor I will do that if there is an influance on the numbers if we will devide the data gender wise**

In [45]:

total_incom_mean_age_inc_gender = credit_df_nomiss.groupby(['age_group','income_type_grade','gender']).agg({'days_employed':['mean','median'],'total_income':['mean','median']})

total_incom_mean_age_inc_gender

days_employed                 \
                                             mean         median   
age_group income_type_grade gender                                 
1         2                 F          452.476523     466.645577   
                            M          627.126553     589.676848   
          3                 F          742.610885     798.219165   
                            M          838.574270     885.674587   
          4                 F          509.969922     509.969922   
                            M          645.671074     645.671074   
2         1                 F          520.848083     520.848083   
          2                 F         1240.274210     992.785520   
                            M         1194.342342     942.921242   
          3                 F         1306.053113    1114.868798   
                            M         1227.441351     975.967849   
          4                 F         1669.439282    1477.522352   
                            M         1778.565761    1652.793945   
          5                 M          578.751554     578.751554   
          6                 F       356755.199027  350340.760224   
                            M       370586.391585  370586.391585   
3         2                 F         1878.660110    1544.319746   
                            M         1933.822105    1487.808833   
          3                 F         2157.260772    1679.420022   
                            M         1936.867183    1416.819709   
          4                 F         2805.813669    2516.427214   
                            M         3305.068227    3098.869195   
          6                 F       365957.980121  364038.971624   
                            M       366543.474037  369132.299748   
          7                 F         3296.759962    3296.759962   
          8                 M       337524.466835  337524.466835   
4         2                 F         2592.359953    1956.697148   
                            M         2344.665326    1915.961625   
          3                 F         3010.335027    2245.147007   
                            M         2370.443455    1662.039719   
          4                 F         4116.746508    3479.340229   
                            M         3728.198769    3324.854938   
          6                 F       365071.941515  366186.875297   
                            M       366216.108454  366777.226752   
          8                 F       395302.838654  395302.838654   
5         2                 F         3009.994325    2256.038809   
                            M         2705.046458    1795.202580   
          3                 F         3551.488259    2479.010604   
                            M         2782.400560    1912.523146   
          4                 F         5057.398535    4177.774233   
                            M         4344.059475    3087.712642   
          6                 F       365043.008470  365614.871306   
                            M       362132.469288  359452.662072   
6         2                 F         4104.128487    2657.348903   
                            M         2880.804727    1913.690402   
          3                 F         4508.291117    3314.894331   
                            M         2744.176534    1835.480848   
          4                 F         4909.363706    3886.288012   
                            M         3153.942517    2876.221697   
          6                 F       365804.028144  366612.864296   
                            M       363281.519352  362685.727407   
7         2                 F         5810.241416    6022.360770   
                            M         2245.558492    2245.558492   
          3                 F         3048.246867    1504.924191   
                            M         7920.082429    7920.082429   
          4                 F         1204.922098    1204.922098   
          6             

**Conclusion**

We need to give attention also to the 'gender' parameter. There are places where the difference between gender under the same age and income type.


In [46]:
#Write a function that we will use for filling in missing values

#the table with the median
total_incom_median_age_inc_gender = credit_df_nomiss.groupby(['age_group','income_type_grade','gender']).agg({'total_income':'median'})

        
#function returning the positional values of the row in the total_incom_mean_age_inc_gender table
def get_pos(row):
    
    pos1= row['age_group']
    pos2= row['income_type_grade']
    pos3= row['gender']
    pos4= row['total_income']
    
    return( pos1 , pos2 , pos3 , pos4)

#returning the value of the total incom median based on the criteria
#values  returned from get_pos function

def total_incom_value(row):
    
   
    age_group, income_type_grade , gender , total_income = get_pos(row)
        
    if str(total_income).lower() == 'nan': #if any record will not be able to change then pass to the next one
        try:
            return (total_incom_median_age_inc_gender['total_income'].loc[(age_group , income_type_grade , gender)])
        except:
            pass
    else:
        return total_income



In [47]:
# Check if it works
print(total_incom_value(credit_df.iloc[20]))
print(total_incom_value(credit_df.iloc[45]))
print(total_incom_value(credit_df.iloc[88]))
print(total_incom_value(credit_df.iloc[65]))

33528.422999999995
17021.747
33099.576
29493.747000000003


After We checked that the function workes and returns the values we needed, We can apply it to the missing values of the 'total_income' column.
I will pass the function to the df using apply() method combined with lambda expression , that the condition on applyin this function will be : the value in 'total_income' column in this index = Nan (null value).
I use it after checkin multiple options to do the same, I found it to be the simpiest.

In [48]:
# Apply it to every row of the original data.
#Using the apply() expression.

credit_df['total_income'] = credit_df.apply(total_incom_value , axis = 1)
credit_df_null_new['total_income'] = credit_df_null_new.apply(total_incom_value , axis = 1)


/tmp/ipykernel_160/3916736894.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_df_null_new['total_income'] = credit_df_null_new.apply(total_incom_value , axis = 1)


Now let us check if we have more missing values in credit_df

In [49]:
credit_df.loc[(credit_df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
5879,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,1,5


Found 1 record that could not be applyed the total income median. So I decided to delete it. It will not effect the data in a substantial way.

In [50]:
credit_df = credit_df.drop(credit_df.loc[(credit_df['total_income'].isna())].index)

Chekcing it there are any more null values in the 'total_income' column

In [51]:
credit_df.loc[(credit_df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group


The missing values in the 'total_income have all been filled.
Let us moove to the mssing values in tha 'days_employed' column.

###  Restoring values in `days_employed`

We will go through the phases that we made in filling the 'total_income', to fill the missing values in the 'days_employed' column. 

In [52]:
#Write a function that we will use for filling in missing values

#the table with the median of the 'days_employed' 
days_employed_median_age_inc_gende = credit_df_nomiss.groupby(['age_group','income_type_grade','gender']).agg({'days_employed':'median'})

        
#function returning the positional values of the row in the total_incom_mean_age_inc_gender table
def get_pos(row):
    
    pos1= row['age_group']
    pos2= row['income_type_grade']
    pos3= row['gender']
    pos4= row['days_employed']
    
    return( pos1 , pos2 , pos3 , pos4)

#returning the value of the total incom median based on the criteria
#values  returned from get_pos function

def days_employed_value(row):
    
   
    age_group, income_type_grade , gender , days_employed = get_pos(row)
        
    if str(days_employed).lower() == 'nan': #if any record will not be able to change then pass to the next one
        try:
            return (days_employed_median_age_inc_gende['days_employed'].loc[(age_group , income_type_grade , gender)])
        except:
            pass
    else:
        return days_employed



In [53]:
# Check that the function works

print(days_employed_value(credit_df.iloc[20]))
print(days_employed_value(credit_df.iloc[45]))
print(days_employed_value(credit_df.iloc[88]))
print(days_employed_value(credit_df.iloc[65]))

1311.6041664289485
1039.4517405766287
373129.7896543124
942.9212415042186


In [54]:
# Apply function to the days_employed and replacing the missing vlues acording to the age_group, on both full data table
#and the filltere one.

credit_df['days_employed'] = credit_df.apply(days_employed_value , axis = 1)
credit_df_null_new['days_employed'] = credit_df_null_new.apply(days_employed_value , axis = 1)

In [55]:
# Check if function worked

credit_df.loc[(credit_df['days_employed'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group


No Missing values remaind in the the 'days_employed' column.

**We have no have atable with no missing values nor duplicates.
It is time to check our hypothisys**


After a check ,we find that the missing vlues have been changed by the criteria, and we can now moove on with the data anlyzing.
Let us just check that there are no more missing values.

In [56]:
# Check the entries in all columns - make sure we fixed all missing values
print(len(credit_df.loc[credit_df.isna().any(axis=1)])) #looking for any missing values. 
                                                        #If there are no missing values, the value will return 0

0


In [57]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21285 entries, 0 to 21285
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   children           21285 non-null  int64  
 1   days_employed      21285 non-null  float64
 2   dob_years          21285 non-null  int64  
 3   education          21285 non-null  object 
 4   education_id       21285 non-null  int64  
 5   family_status      21285 non-null  object 
 6   family_status_id   21285 non-null  int64  
 7   gender             21285 non-null  object 
 8   income_type        21285 non-null  object 
 9   debt               21285 non-null  int64  
 10  total_income       21285 non-null  float64
 11  purpose            21285 non-null  object 
 12  income_type_grade  21285 non-null  int64  
 13  age_group          21285 non-null  int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 2.4+ MB


In [58]:
credit_df['days_employed'] = credit_df['days_employed'].astype('int64')
credit_df['total_income'] = credit_df['total_income'].astype('int64') 

In [59]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21285 entries, 0 to 21285
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   children           21285 non-null  int64 
 1   days_employed      21285 non-null  int64 
 2   dob_years          21285 non-null  int64 
 3   education          21285 non-null  object
 4   education_id       21285 non-null  int64 
 5   family_status      21285 non-null  object
 6   family_status_id   21285 non-null  int64 
 7   gender             21285 non-null  object
 8   income_type        21285 non-null  object
 9   debt               21285 non-null  int64 
 10  total_income       21285 non-null  int64 
 11  purpose            21285 non-null  object
 12  income_type_grade  21285 non-null  int64 
 13  age_group          21285 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 2.4+ MB


In [60]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21285 entries, 0 to 21285
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   children           21285 non-null  int64 
 1   days_employed      21285 non-null  int64 
 2   dob_years          21285 non-null  int64 
 3   education          21285 non-null  object
 4   education_id       21285 non-null  int64 
 5   family_status      21285 non-null  object
 6   family_status_id   21285 non-null  int64 
 7   gender             21285 non-null  object
 8   income_type        21285 non-null  object
 9   debt               21285 non-null  int64 
 10  total_income       21285 non-null  int64 
 11  purpose            21285 non-null  object
 12  income_type_grade  21285 non-null  int64 
 13  age_group          21285 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 2.4+ MB


# 5  Categorization of data

We are now with clean data with no missing or duplicated records.

To address the question , if the merital status and having children , influance the abilty of a customer, to return the loan.

Because this is the question, we will need the columns : 'children' , 'family_status' ,'family_status_id' and 'debt' .

But since this are not the only categories that can influance the default of a loan.
The age of the customer when applying for a loan, can influance. 
The income can also influance it.
But the most influancing is the amount you take in the loan.
This is a factor of the purpose of taking the loan. 

If we will see that there is no corrolation between the marital status and having children, to a default of a loan, we will check the loan purpose.
For that we will need to categorise the data in the 'purpose' column.

We will start by categorizing the 'purpose' column.

And then start the investigation of the data and check our hypotheses.


In [61]:
#let us check the unique values of the 'purpose' column, and see how we can categorize it.
lst_purpose = list(credit_df['purpose'].unique())
lst_purpose = sorted(lst_purpose)
lst_purpose

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

**The main purposes I can identify are:**

Buy a house/investing in a property, education , car and wedding.

To categorize the column we will build a few lists with few key words, that will give us the base on identifying the category to which this value will be assigned. 

The four categories in this column will be:

property , education ,car , wedding



In [62]:
#Building a set of lists containing key words for each category.
lst_property = ['property', 'estate' , 'housing' , 'house']
lst_car = ['car', 'cars']
lst_education = ['education', 'university' , 'educated']
lst_wedding = ['wedding']

#define a function that will check each value in purpose and return its category
def purpose_category(purpose):
    
    
    lst_purpose = purpose['purpose'].split(" ")
    
    for str_purpose in lst_purpose:
        
        if str_purpose in lst_property:
            return ('property')
        elif str_purpose in lst_car:
            return ('car')
        elif str_purpose in lst_education:
            return ('education')
        elif str_purpose in lst_wedding:
            return ('wedding')

#applying the function to the 'purpose' column
credit_df['purpose_category'] = credit_df.apply(purpose_category , axis = 1)

credit_df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,3,4,property
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,3,3,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,3,3,property
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,3,3,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,6,5,wedding


In [63]:
#check the function
print(credit_df['purpose'].iloc[10])
print(purpose_category(credit_df.iloc[10]))
print(credit_df['purpose'].iloc[85])
print(purpose_category(credit_df.iloc[85]))
print(credit_df['purpose'].iloc[60])
print(purpose_category(credit_df.iloc[60]))
print(credit_df['purpose'].iloc[89])
print(purpose_category(credit_df.iloc[89]))

buy real estate
property
university education
education
to become educated
education
second-hand car purchase
car


In [64]:
#applying the function to the 'purpose' column
credit_df['purpose_category'] = credit_df.apply(purpose_category , axis = 1)

credit_df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,3,4,property
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,3,3,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,3,3,property
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,3,3,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,6,5,wedding


**We need to check for any missing values in the new column - to find out is al values were inserted**

In [65]:
#count the number of values with missing values - if 0 , all values were entered to each category.
credit_df.loc[(credit_df['purpose_category'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category


The 'pupose' column now has its categories. We can moove to the next step.

Most of the colums are now categorized. Let us group the data by the criteria we wanted to examin.

We have categorized every column we need to check our hypotheses.
Now we need to preper the data for analysis. Building new customized tables based on the criteria we chose before, will make it easy to check the data.

The first table will be the one that will combine the columns that have the marital status, number of children and the debt ststus , for each customer in the data set. This table will give me the data I need to check the hypotheses.

Then We will group the data of the total income and the loan purpose with the debt status and see the resault we get when manipulatin the data.

**Now we start with the first table cobination**

In [66]:
# Print the values for your selected data of the criteria that wil help us check the hypotyses we want to examin.

#selecting the relevent columns to a new filtered table

credit_df_children_marital_status = credit_df[['children' , 'family_status' , 'family_status_id','debt']]

credit_df_children_marital_status.head(20)

,children,family_status,family_status_id,debt
0,1,married,0,0
1,1,married,0,0
2,0,married,0,0
3,3,married,0,0
4,0,civil partnership,1,0
5,0,civil partnership,1,0
6,0,married,0,0
7,0,married,0,0
8,2,civil partnership,1,0
9,0,married,0,0


## Checking the Hypotheses


We will first filter the data of the categories we want to check first (merital status , no. of children and the debt) , so the the new filltered table will contain only the records of the customers who have debt.

In [67]:
#creating a filltered table from credit_df_children_marital_status_filltered table 
#containing only the customeres that have debt

credit_df_children_marital_status_filltered = credit_df_children_marital_status.loc[(credit_df_children_marital_status['debt'] == 1)]

#reseting the indexes

credit_df_children_marital_status_filltered = credit_df_children_marital_status_filltered.reset_index(drop=True)

#print the first 15 rows of the filltered table

credit_df_children_marital_status_filltered.head(15)

,children,family_status,family_status_id,debt
0,0,civil partnership,1,1
1,0,civil partnership,1,1
2,0,unmarried,4,1
3,0,civil partnership,1,1
4,1,married,0,1
5,1,married,0,1
6,0,widow / widower,2,1
7,0,married,0,1
8,0,married,0,1
9,0,married,0,1


Let us make a table with the calculations of the count of debt by no. of children under each marital status from filltered  data . and see if there is a uniqe distribustion of values where there is a debt.

In [68]:
#Grouping the data of the children and marital status where there is debt. 
#and calculating each the sum of them, the count of records, the mean and media of these vlues.from the majority

credit_df_children_marital_status_filltered_grouped  = credit_df_children_marital_status_filltered.groupby(['family_status','children'])['debt'].agg(Count='count')

#print the resault
credit_df_children_marital_status_filltered_grouped

Count
family_status     children       
civil partnership 0           227
                  1           118
                  2            33
                  3             8
divorced          0            55
                  1            21
                  2             8
                  3             1
married           0           514
                  1           245
                  2           148
                  3            17
                  4             3
unmarried         0           210
                  1            51
                  2            10
                  3             1
                  4             1
widow / widower   0            52
                  1             7
                  2             3

Now let us add the rate of the data for each resault

In [69]:
#adding  a new rate column - the percent of values from the whole data.

credit_df_children_marital_status_filltered_grouped['rate_of_debt'] = credit_df_children_marital_status_filltered_grouped['Count'] / len(credit_df_children_marital_status_filltered) * 100

#print the resaults
credit_df_children_marital_status_filltered_grouped


Count  rate_of_debt
family_status     children                     
civil partnership 0           227     13.098673
                  1           118      6.809002
                  2            33      1.904212
                  3             8      0.461627
divorced          0            55      3.173687
                  1            21      1.211771
                  2             8      0.461627
                  3             1      0.057703
married           0           514     29.659550
                  1           245     14.137334
                  2           148      8.540104
                  3            17      0.980958
                  4             3      0.173110
unmarried         0           210     12.117715
                  1            51      2.942874
                  2            10      0.577034
                  3             1      0.057703
                  4             1      0.057703
widow / widower   0            52      3.000577
                  1             7      0.403924
                  2             3      0.173110

Let us sea the general data distribution of these indexes

In [70]:
#making a new table for the general data of these columns

credit_df_children_marital_status_grouped  = credit_df_children_marital_status.groupby(['family_status','children'])['debt'].agg(Count='count')

#calculatin the rate of the general data of these criteria.
credit_df_children_marital_status_grouped['rate_of_debt'] = credit_df_children_marital_status_grouped['Count'] / len(credit_df) * 100

#Join the resaults to figure out if the hypotheses is correct.
credit_df_children_marital_status_general_join = credit_df_children_marital_status_filltered_grouped.join(credit_df_children_marital_status_grouped, lsuffix='_debt', rsuffix='_general')

#print the resault
credit_df_children_marital_status_general_join


Count_debt  rate_of_debt_debt  Count_general  \
family_status     children                                                 
civil partnership 0                227          13.098673           2697   
                  1                118           6.809002            991   
                  2                 33           1.904212            352   
                  3                  8           0.461627             56   
divorced          0                 55           3.173687            777   
                  1                 21           1.211771            315   
                  2                  8           0.461627             81   
                  3                  1           0.057703             11   
married           0                514          29.659550           7406   
                  1                245          14.137334           2990   
                  2                148           8.540104           1571   
                  3                 17           0.980958            247   
                  4                  3           0.173110             29   
unmarried         0                210          12.117715           2246   
                  1                 51           2.942874            452   
                  2                 10           0.577034             83   
                  3                  1           0.057703              8   
                  4                  1           0.057703              2   
widow / widower   0                 52           3.000577            841   
                  1                  7           0.403924             81   
                  2                  3           0.173110             24   

                            rate_of_debt_general  
family_status     children                        
civil partnership 0                    12.670895  
                  1                     4.655861  
                  2                     1.653747  
                  3                     0.263096  
divorced          0                     3.650458  
                  1                     1.479915  
                  2                     0.380550  
                  3                     0.051680  
married           0                    34.794456  
                  1                    14.047451  
                  2                     7.380785  
                  3                     1.160442  
                  4                     0.136246  
unmarried         0                    10.552032  
                  1                     2.123561  
                  2                     0.389946  
                  3                     0.037585  
                  4                     0.009396  
widow / widower   0                     3.951139  
                  1                     0.380550  
                  2                     0.112755

**Conclusion**

Looking at the rates we see the opposit effect that we first hypothesised. 
We can see that the most records of people that have a debt is cossistent to their percent of the general population.

From the numbers we can not conclude that there is a direct corralation between having children to having a debt. There is a consistency of the part of the customers that have or not children to theire general part of the population. And it make sense that there will be a cosistancy of debt based that corralation.


**The Next step - Checking the other Hypotheses**

How the purpose of the loan effects the default of the loan.

before we do that , we need to categorize the income column of the filtered table.There are too many salleries with diffrent total sum of them so we can not group the data like that.And if we will need to enter the incom factor the we will need to categorize it any way.

In [71]:
#new table sorted by income
credit_df_sorted_incom = credit_df.sort_values(by='total_income' , ascending = True , ignore_index = True)
#printing the first 15 rows to get the first lowest value in total_income
#givving us the low border of the categories we want to apply
credit_df_sorted_incom.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category
0,0,359219,57,secondary education,1,married,0,F,retiree,1,3306,property,6,5,property
1,0,369708,37,secondary education,1,civil partnership,1,M,retiree,0,3392,going to university,6,3,education
2,1,3642,52,secondary education,1,married,0,M,employee,0,3418,car purchase,3,5,car
3,0,359726,68,secondary education,1,civil partnership,1,M,retiree,0,3471,having a wedding,6,6,wedding
4,0,346602,61,secondary education,1,married,0,F,retiree,0,3503,property,6,6,property


In [72]:
#print some middle rows to see the range of values we get there
credit_df_sorted_incom.loc[1000:1015]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category
1000,0,357,48,secondary education,1,widow / widower,2,F,business,0,10195,purchase of the house for my family,2,4,property
1001,0,1051,28,secondary education,1,married,0,M,employee,0,10199,purchase of a car,3,2,car
1002,0,344108,69,secondary education,1,widow / widower,2,M,retiree,0,10200,housing,6,6,property
1003,0,2351,30,secondary education,1,married,0,M,employee,0,10200,university education,3,2,education
1004,0,346047,54,secondary education,1,married,0,F,retiree,0,10203,purchase of my own house,6,5,property
1005,0,8797,46,secondary education,1,married,0,F,employee,0,10205,purchase of the house,3,4,property
1006,0,3167,41,secondary education,1,married,0,F,employee,0,10205,purchase of the house for my family,3,4,property
1007,1,247,38,secondary education,1,married,0,F,business,0,10216,purchase of the house,2,3,property
1008,0,11966,52,secondary education,1,civil partnership,1,F,employee,0,10217,purchase of my own house,3,5,property
1009,0,3766,30,secondary education,1,married,0,F,business,0,10219,housing transactions,2,2,property


In [73]:
#printing the last rows to see the highest border of the  total income
credit_df_sorted_incom.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category
21280,0,5734,42,bachelor's degree,0,civil partnership,1,M,business,0,273809,to have a wedding,2,4,wedding
21281,0,4719,61,secondary education,1,unmarried,4,F,employee,0,274402,purchase of the house for my family,3,6,property
21282,1,5248,35,secondary education,1,civil partnership,1,M,employee,0,276204,supplementary education,3,3,education
21283,1,2577,39,bachelor's degree,0,married,0,M,business,1,352136,building a property,2,3,property
21284,0,1477,44,bachelor's degree,0,married,0,M,business,0,362496,housing renovation,2,4,property


**getting the income values from the filltered table**

let us do the same with the filltered table. what incom values do we have there?

In [74]:
credit_df_null_new = credit_df_null_new.reset_index(drop=True)
credit_df_null_new.sort_values(by='total_income', ascending=True).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
329,0,350340.760224,24,secondary education,1,married,0,F,retiree,0,10431.696,purchase of my own house,6,2
1578,0,798.219165,20,secondary education,1,civil partnership,1,F,employee,0,15296.589,housing,3,1
937,1,798.219165,20,secondary education,1,civil partnership,1,F,employee,0,15296.589,buying a second-hand car,3,1
1743,1,369733.465222,73,secondary education,1,married,0,F,retiree,0,17757.684,supplementary education,6,7
951,0,369733.465222,72,secondary education,1,unmarried,4,F,retiree,1,17757.684,car purchase,6,7


In [75]:
credit_df_null_new.sort_values(by='total_income', ascending=True).tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group
1643,0,1915.961625,41,bachelor's degree,0,civil partnership,1,M,business,0,33594.343,wedding ceremony,2,4
1044,1,1915.961625,45,bachelor's degree,0,married,0,M,business,0,33594.343,getting higher education,2,4
1107,1,1915.961625,46,secondary education,1,married,0,M,business,0,33594.343,construction of own property,2,4
1645,0,1915.961625,49,some college,2,married,0,M,business,0,33594.343,getting an education,2,4
532,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,1,5


**Categorizing the total_income values**

By categorizing the total_income column , we can make groups that are  enabling investigation by income ratings , that gives us a good assesment on the customer income status.
we will categorize to 10 catecorizations:

1 : 3000 - 15000

2 : 15000 - 30000

3 : 30000 - 45000

4 : 45000 - 80000

5 : 80000 - 150000

6 : 150000 - 250000

7 : 250000+

this ia an arbituary divission basd on the borders of the total_income column in the filltered table.

In [76]:
#creatin a categirizin function

def get_income_category(row):
    
    income = row['total_income']
    
    
    if income >=3000 and income <15000:
        return 1
    elif income >=15000 and income <30000:
        return 2
    elif income >=30000 and income <45000:
        return 3
    elif income >=45000 and income <80000:
        return 4
    elif income >=80000 and income <150000:
        return 5
    elif income >=150000 and income <250000:
        return 6
    elif income >=250000:
        return 7
    
# apply the function to the 'total_income' column creating a nre category column
credit_df['income_category'] = credit_df.apply( get_income_category , axis =1 )

#printing the firs 15 rows 
credit_df.head(15)



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_type_grade,age_group,purpose_category,income_category
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,3,4,property,3
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,3,3,car,2
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,3,3,property,2
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,3,3,education,3
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,6,5,wedding,2
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922,purchase of the house,2,2,property,3
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484,housing transactions,2,4,property,3
7,0,152,50,secondary education,1,married,0,M,employee,0,21731,education,3,4,education,2
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337,having a wedding,3,3,wedding,2
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108,purchase of the house for my family,3,4,property,2


**We  can now check the next hypotheses**

**Does the puprpose of the loan effects its default**

we will check it with cobining the income category we made.

In [77]:
#creating a table with the columns we wanted
credit_df_income_purpose = credit_df[['purpose_category','debt']]

#creating a filltered table from credit_df_children_marital_status_filltered table 
#containing only the customeres that have debt

credit_df_income_purpose_filltered = credit_df_income_purpose.loc[(credit_df_children_marital_status['debt'] == 1)]

#reseting the indexes

credit_df_income_purpose_filltered = credit_df_income_purpose_filltered.reset_index(drop=True)

#print the first 15 rows of the filltered table

credit_df_income_purpose_filltered.head(15)

,purpose_category,debt
0,property,1
1,wedding,1
2,education,1
3,wedding,1
4,property,1
5,car,1
6,property,1
7,property,1
8,property,1
9,property,1


In [78]:
#Grouping the data of the purpose where there is debt. 
#and count of records
credit_df_filltered = credit_df.loc[credit_df['debt'] == 1]

credit_df_income_purpose_filltered_grouped  = credit_df_filltered.groupby(['purpose_category'])['debt'].agg(Count='count')

#adding  a new rate column - the percent of values from the whole data.

credit_df_income_purpose_filltered_grouped['rate_of_debt'] = credit_df_income_purpose_filltered_grouped['Count'] / len(credit_df_income_purpose_filltered) * 100

#making a new table for the general data of these columns

credit_df_income_purpose_general_grouped  = credit_df_income_purpose.groupby(['purpose_category'])['debt'].agg(Count='count')

#calculatin the rate of the general data of these criteria.
credit_df_income_purpose_general_grouped['rate_of_debt'] = credit_df_income_purpose_general_grouped['Count'] / len(credit_df) * 100

#Join the resaults to figure out if the hypotheses is correct.
credit_df_income_purpose_joined = credit_df_income_purpose_filltered_grouped.join(credit_df_income_purpose_general_grouped, lsuffix='_debt', rsuffix='_general')

#print the resault
credit_df_income_purpose_joined



,Count_debt,rate_of_debt_debt,Count_general,rate_of_debt_general
purpose_category,,,,
car,400,23.081362,4275,20.084567
education,370,21.350260,3986,18.726803
property,779,44.950952,10735,50.434578
wedding,184,10.617426,2289,10.754052


**Intermediate Conclusion**

we have a stagaring number of 50% of the customers who applied for a loan to buy a hous/ renovate one/ invest in a property , are in debt. This is a very high rate for the default of a loan.But when we look at the numbers of loan, we see that it is only in a rate of 7% approx.

further investigation is needed.

In [80]:
#checking the purpose category
credit_df.groupby('purpose_category')['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

,Count,Sum,Mean
purpose_category,,,
car,4275,400,0.093567
education,3986,370,0.092825
property,10735,779,0.072566
wedding,2289,184,0.080384


    **The most percent of people that have debt are the one from the car purchases and second are the education loans.**

In [81]:
#checking the age category
credit_df.groupby('age_group')['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

,Count,Sum,Mean
age_group,,,
1,65,5,0.076923
2,3645,398,0.109191
3,5721,546,0.095438
4,5245,403,0.076835
5,4495,280,0.062291
6,2011,98,0.048732
7,103,3,0.029126


    **The most percent of people that have debt are the ones who are in age group 2 - 20-30 yo.
    behind them in a very close proximity is the 30-40 age group.**

In [82]:
#checking the age category
credit_df.groupby('gender')['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

,Count,Sum,Mean
gender,,,
F,14044,989,0.070422
M,7241,744,0.102748


    **The most percent of people that have debt are the ones who are male group **

In [83]:
#checking the income category
credit_df.groupby('income_category')['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

,Count,Sum,Mean
income_category,,,
1,3724,297,0.079753
2,11521,990,0.085930
3,4188,314,0.074976
4,1630,118,0.072393
5,194,12,0.061856
6,22,1,0.045455
7,6,1,0.166667


    **The most percent of people that have debt are the one from the 7 income group. meaning over 250000 a year
     But the count of these people that took a loan are 6 so we can not include it as a valid statement.
     The Second income grade that is with the higher percent is no. 2 income category - 15000-30000 anualy income. **

**Now we should try to waht happans when we combine 2 categories**

In [84]:
#purpose combine with age
credit_df.groupby(['purpose_category','age_group'])['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

Count  Sum      Mean
purpose_category age_group                      
car              1             15    1  0.066667
                 2            670   83  0.123881
                 3           1164  138  0.118557
                 4           1069   91  0.085126
                 5            918   62  0.067538
                 6            423   23  0.054374
                 7             16    2  0.125000
education        1              9    0  0.000000
                 2            710   93  0.130986
                 3           1055  112  0.106161
                 4            950   82  0.086316
                 5            848   62  0.073113
                 6            395   21  0.053165
                 7             19    0  0.000000
property         1             35    4  0.114286
                 2           1880  184  0.097872
                 3           2868  240  0.083682
                 4           2669  182  0.068190
                 5           2243  124  0.055283
                 6            983   44  0.044761
                 7             57    1  0.017544
wedding          1              6    0  0.000000
                 2            385   38  0.098701
                 3            634   56  0.088328
                 4            557   48  0.086176
                 5            486   32  0.065844
                 6            210   10  0.047619
                 7             11    0  0.000000

**the consistancy is the same when checking age and purpose**

In [85]:
#purpose combine with gender
credit_df.groupby(['purpose_category','gender'])['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

Count  Sum      Mean
purpose_category gender                      
car              F        2833  232  0.081892
                 M        1442  168  0.116505
education        F        2644  207  0.078290
                 M        1342  163  0.121461
property         F        7034  443  0.062980
                 M        3701  336  0.090786
wedding          F        1533  107  0.069798
                 M         756   77  0.101852

**the consistancy is the same when checking age and purpose**

In [86]:
#purpose combine with income
credit_df.groupby(['purpose_category','income_category'])['debt'].agg(Count='count', Sum='sum', Mean = 'mean')

Count  Sum      Mean
purpose_category income_category                      
car              1                  736   62  0.084239
                 2                 2316  231  0.099741
                 3                  848   78  0.091981
                 4                  335   26  0.077612
                 5                   37    3  0.081081
                 6                    3    0  0.000000
education        1                  718   61  0.084958
                 2                 2195  220  0.100228
                 3                  744   64  0.086022
                 4                  295   21  0.071186
                 5                   30    3  0.100000
                 6                    3    1  0.333333
                 7                    1    0  0.000000
property         1                 1873  136  0.072611
                 2                 5765  431  0.074761
                 3                 2142  146  0.068161
                 4                  828   60  0.072464
                 5                  108    5  0.046296
                 6                   15    0  0.000000
                 7                    4    1  0.250000
wedding          1                  397   38  0.095718
                 2                 1245  108  0.086747
                 3                  454   26  0.057269
                 4                  172   11  0.063953
                 5                   19    1  0.052632
                 6                    1    0  0.000000
                 7                    1    0  0.000000

# General Conclusion 

The question that was laid to our hands was , if a merital and having children, can be considered a a default of a loan reason more then other reasons.

When put to the test, with both of these criteria, we have not found any diviation of the general population of both having a child, nor the merital status of a customer, in records of customers who have debt.

There for we could get to the conclusion tha the merital status and/or having children, have an effect on default a loan, at least no a profaund efect that shows in the data.

But when looking at other possible reasons, one criteria has stood up showing that there are categories from it that are shown in a very concentrated way in the group of customers who have debt. This criteria was the 'purpose' category of 'car' loan application.

Also , It seems that when the total income of a customer is less then 30,000 anualy and he is applying for loan, has more chance to default.

In conclusion*

we did not find any connectio between tha merital status and having children , direct effect on a loan default. Instead, we sugest that the criteria to be taken is the one of purpose of a loan that is intended to be use for a car , and to look at the total income of a customer that earns less then 30,000 anualy.